# 目的
PyFoamでパラメータスタディを行い、PyVistaで画像出力を行う
- オリジナルケースは```orgCase```フォルダを準備する

## プログラムの流れ
- resultsフォルダを作成
- ```orgCase```をresultsフォルダにコピーする
- 001_cavutyの流速を変更
- 001_cavutyの計算実行
- 002_particleへ移動
- 001_cavityの流速を002_particleへマッピング
- 粒子径を変更する
- 計算を実行する（```Allrun```）スクリプトを実行する
- PyVistaでresultsフォルダの中に```.png```画像ファイルを作成

```orgCase```のAllrunスクリプトを以下としている。

### Allrunスクリプト
```
#!/bin/bash

# cavityの計算
cd 001_cavity
./Allrun.icoFoam

# 粒子追跡の計算
cd ..
cd 002_particle
./Allrun.icoUncoupledKinematicParcelFoam
```

パラスタを行う前に必ず```Allrun```スクリプトはオリジナルケース```orgCase```で実行して正常に動作することを確認すること。

### 001_cavityのAllrun.icoFoamのスクリプト
```
#!/bin/bash

#blockMesh
icoFoam
```

### 002_particleのAllrun.icoUncoupledKinematicParcelFoamのスクリプト
```
#!/bin/bash

mapFields ../001_cavity -sourceTime 0.5 -consistent
icoUncoupledKinematicParcelFoam
foamToVTK
```

# PyFoamを使ったパラスタのプログラム（画像出力はPyVista）

In [1]:
import os
import shutil
import threading
from pathlib import Path
import subprocess
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.Execution.BasicRunner import BasicRunner

def clone_file(orgCase, resultDir):
    "orgCaseをresultDirに重複しないようにコピー"
    baseName = Path(orgCase).name
 
    addName = None
    if addName!=None:
        baseName += f'_{addName}'
 
    Path(resultDir).mkdir(exist_ok=True, parents=True)
 
    n=0
    newCase = Path(resultDir) / f'{baseName}_{n}'
 
    while newCase.is_dir():
        n += 1
        newCase = Path(resultDir) / f'{baseName}_{n}'
 
    shutil.copytree(orgCase, newCase)
    
    return newCase

orgCase = "orgCase"
resultDir = "results"

U_list = [0.5, 1.0, 2.0, 3.0, 4.0, 5.0]
d_mean_list = [0.005, 0.01, 0.02, 0.03, 0.04, 0.05]

for U in U_list:
    for d_mean in d_mean_list:
        # 新しいファイルをコピー
        newCase = clone_file(orgCase, resultDir)
    
        # 流速の変更(001_cavity)
        cavity_dir = f"{newCase}/001_cavity"
        UFile = ParsedParameterFile(os.path.join(cavity_dir, "0", "U"))
        UFile["boundaryField"]["movingWall"]["value"] = f'uniform ({U} 0 0)'
        
        # 粒子径の変更 (002_particle)
        particle_dir = f"{newCase}/002_particle"
        kinematicCloudPropertiesFile = ParsedParameterFile(os.path.join(particle_dir, "constant", "kinematicCloudProperties"))
        kinematicCloudPropertiesFile["subModels"]["injectionModels"]["model1"]["sizeDistribution"]["fixedValueDistribution"]["value"] = d_mean
        kinematicCloudPropertiesFile.writeFile()
        print(f"{newCase}：全てのクローン操作とファイル編集が完了しました。")
        
        with open(os.devnull, 'w') as devnull:
            subprocess.run([f'./Allrun'], cwd=newCase, stdout=devnull, stderr=devnull)

results/orgCase_0：全てのクローン操作とファイル編集が完了しました。
results/orgCase_1：全てのクローン操作とファイル編集が完了しました。
results/orgCase_2：全てのクローン操作とファイル編集が完了しました。
results/orgCase_3：全てのクローン操作とファイル編集が完了しました。
results/orgCase_4：全てのクローン操作とファイル編集が完了しました。
results/orgCase_5：全てのクローン操作とファイル編集が完了しました。
results/orgCase_6：全てのクローン操作とファイル編集が完了しました。
results/orgCase_7：全てのクローン操作とファイル編集が完了しました。
results/orgCase_8：全てのクローン操作とファイル編集が完了しました。
results/orgCase_9：全てのクローン操作とファイル編集が完了しました。
results/orgCase_10：全てのクローン操作とファイル編集が完了しました。
results/orgCase_11：全てのクローン操作とファイル編集が完了しました。
results/orgCase_12：全てのクローン操作とファイル編集が完了しました。


KeyboardInterrupt: 

In [34]:
import pyvista as pv

# PyVistaでの処理
# ファイルパス
particle_dir = 'results/orgCase_8/002_particle'
filename = f"{particle_dir}/VTK/lagrangian/kinematicCloud/kinematicCloud_5000.vtp"

# メッシュデータの読み込み
vtk_mesh = pv.read(filename)

# ベクトルを矢印で表示
glyphs = vtk_mesh.glyph(orient="U", scale="U", factor=0.03)

# プロッターを作成
plotter = pv.Plotter(off_screen=True)  # GUI を開かずにオフスクリーン描画

# メッシュを追加
plotter.add_mesh(vtk_mesh, color="blue", point_size=30, render_points_as_spheres=True)
plotter.add_mesh(glyphs, cmap="jet")

# カメラ調整（オプション）
plotter.camera_position = "xy"
# plotter.zoom_camera(10.0)

plotter.show()
# 画像を保存
# plotter.screenshot(f"{resultDir}/{d_mean}.png")

# print("PNG画像として保存しました: output.png")

Widget(value='<iframe src="http://localhost:44411/index.html?ui=P_0x7f1cc85f3130_7&reconnect=auto" class="pyvi…